In [1]:
pip install osmnx geopandas matplotlib shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.3/101.3 kB 2.5 MB/s eta 0:00:00


In [2]:
pip install alphashape

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.1/541.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.8/708.8 kB 13.3 MB/s eta 0:00:00


In [3]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
import time
from shapely.geometry import Point, Polygon, MultiPoint
import numpy as np
from scipy.spatial import Delaunay  # Правильный импорт Delaunay
import math



In [4]:
ox.settings.log_console = True
ox.settings.use_cache = True
ox.settings.timeout = 600  # Увеличим таймаут для Москвы

# Загрузка пешеходной сети для Москвы
print("Загрузка дорожной сети Москвы...")
place_name = "Moscow, Russia"
G = ox.graph_from_place(place_name, network_type='walk')


Загрузка дорожной сети Москвы...


In [5]:
import pickle

In [6]:
with open('moscow_walk_graph.pkl', 'wb') as f:
    pickle.dump(G, f)

In [7]:
GRAPH_FILENAME = "moscow_walk_graph.graphml"
ox.save_graphml(G, GRAPH_FILENAME)
print(f"Граф сохранен в файл: {GRAPH_FILENAME}")

Граф сохранен в файл: moscow_walk_graph.graphml


In [ ]:
import alphashape

In [8]:
# Изохрона на 3 минуты
#iso_time = 3


def create_isochrone(latitude, longitude, walking_speed, iso_time):
    # Выбор начальной точки (на рандом ткнул)
    #latitude, longitude = 55.762616, 37.613106
    #print(f"Начальная точка: {latitude}, {longitude}")
    start_node = ox.distance.nearest_nodes(G, longitude, latitude)

    edge_lengths = nx.get_edge_attributes(G, 'length')

    # Создаем взвешенный граф для расчета времени
    for u, v, k in G.edges(keys=True):
        length = edge_lengths.get((u, v, k), 0)
        # Время в минутах: расстояние(м) / (скорость(км/ч) * 1000 / 60)
        G[u][v][k]['travel_time'] = length / (walking_speed * 1000 / 60)

    # Рассчитываем время пути
    travel_times_min = nx.single_source_dijkstra_path_length(
        G,
        start_node,
        weight='travel_time'
    )

    reached_nodes = [node for node, time in travel_times_min.items() if time <= iso_time]
    # Создаем GeoDataFrame с достижимыми точкам
    points = [Point(G.nodes[node]['x'], G.nodes[node]['y']) for node in reached_nodes]
    gdf_nodes = gpd.GeoDataFrame(geometry=points, crs="EPSG:4326")

    gdf_nodes_utm = gdf_nodes.to_crs(gdf_nodes.estimate_utm_crs())
    points_utm = [(point.x, point.y) for point in gdf_nodes_utm.geometry]

    # Создаем полигон изохроны
    #print("Создание полигона изохроны...")
    #print("from", points_utm)
    iso_polygon_utm = MultiPoint(points_utm).convex_hull
    #------------------iso_polygon_utm = alpha_shape(points_utm)#_utm, alpha=0.01  # alpha регулирует детализацию
    #iso_polygon_wgs = gpd.GeoSeries(iso_polygon_utm, crs=gdf_nodes_utm.crs).to_crs(4326).iloc[0]

    #Мы берем CRS из GeoDataFrame с точками в UTM.
    iso_gdf_utm = gpd.GeoDataFrame(geometry=[iso_polygon_utm], crs=gdf_nodes_utm.crs)
    iso_gdf = iso_gdf_utm.to_crs("EPSG:4326")

    #return iso_gdf
    filename = f"moscow_{walking_speed}min_isochrone{latitude}_{longitude}.geojson"
    iso_gdf.to_file(filename, driver="GeoJSON")
    print(f"Изохрона сохранена в: {filename}")

    # Сохраняем изохрону в GeoJSON
    #iso_gdf = gpd.GeoDataFrame(geometry=[iso_polygon_utm], crs="EPSG:4326")
    #iso_gdf.to_file(f"moscow_{walking_speed}min_isochrone{latitude}_{longitude}.geojson", driver="GeoJSON")
    #print(f"Изохрона сохранена в: moscow_{walking_speed}min_isochrone{latitude}_{longitude}.geojson")


In [9]:
create_isochrone(55.756862, 37.588718, 4.5, 10)

Изохрона сохранена в: moscow_4.5min_isochrone55.756862_37.588718.geojson


In [ ]:
# Создаем интерактивную карту
latitude, longitude = 55.756862, 37.588718
iso_time = 10
walking_speed = 4.5
print("Создание интерактивной карты...")
m = folium.Map(location=[latitude, longitude],
               zoom_start=17,
               tiles='cartodbpositron')

# Добавляем изохрону
#iso_gdf = gpd.read_file(f"moscow_{walking_speed}min_isochrone{latitude}_{longitude}.geojson")
iso_gdf = gpd.read_file(f"moscow_{walking_speed}min_isochrone{latitude}_{longitude}.geojson")
#iso_gdf = iso_gdf.to_crs("EPSG:4326")

folium.GeoJson(
    iso_gdf,
    style_function=lambda x: {
        'fillColor': '#1abc9c',
        'color': '#16a085',
        'weight': 3,
        'fillOpacity': 0.45
    },
    tooltip=f'{iso_time} min walk area'
).add_to(m)

# Добавляем стартовую точку
folium.Marker(
    [latitude, longitude],
    icon=folium.Icon(color='red', icon='star', prefix='fa'),
    tooltip='Start: Red Square'
).add_to(m)

# Добавляем дорожную сеть (упрощенную)
print("Добавление дорожной сети на карту...")
edges = ox.graph_to_gdfs(G, nodes=False)
for _, row in edges.sample(500).iterrows():
    folium.PolyLine(
        locations=[(point[1], point[0]) for point in row['geometry'].coords],
        color='#7f8c8d',
        weight=1,
        opacity=0.3
    ).add_to(m)

# Сохраняем карту
m.save(f'moscow_{walking_speed}min_isochrone{latitude}_{longitude}.html')
print(f"Интерактивная карта сохранена в: moscow_{walking_speed}min_isochrone{latitude}_{longitude}.html")
print("Откройте файл в браузере для просмотра")

Создание интерактивной карты...
Добавление дорожной сети на карту...


In [ ]:
# прочитать датафрейм
import pandas as pd

In [ ]:
points_df = pd.read_csv("all_cafes_cleaned.csv")
points_df

,name,lat,lon,address,rating,tags,source
0,Школьник,55.761731,37.656844,"улица Земляной Вал, 12/7 ст1, 1 этаж, Москва",2.9,кофейня,Zoon
1,Coffeemolka,55.701928,37.765984,"улица Зеленодольская, 45 корпус 1, Москва",4.2,кофейня,Zoon
2,Шоколадница,55.980913,37.410896,"Международный Аэропорт Шереметьево, терминал B...",3.0,кофейня,Zoon
3,Кофе Хауз,55.755937,37.628088,"Ильинка, 13/19, стр. 2, Москва",3.7,"кондитерская, кофейня",Zoon
4,Шоколадница,55.863789,37.544854,"Дмитровское шоссе, 89, в ТЦ ""XL"", 1 этаж, Москва",2.9,кофейня,Zoon
...,...,...,...,...,...,...,...
1349,chetverg,55.794601,37.607904,NaN,NaN,"V60, Dog friendly, Specialty cacao, Пекарня",Coffeemap
1350,chehov-i-kompaniya,55.742451,37.649601,NaN,NaN,"V60, Декаф, Dog friendly, Laptop friendly",Coffeemap
1351,shkolnik-kofe,55.761769,37.656632,NaN,NaN,"V60, Продажа зерна, Dog friendly, Specialty cacao",Coffeemap
1352,eklernaya-kler,55.770496,37.633162,NaN,NaN,Dog friendly,Coffeemap


In [ ]:
points_df = pd.read_csv("all_cafes_cleaned.csv")
SPEEDS_KMH = [4.5, 12]  # Скорость пешком и на самокате/легком беге
ISO_TIMES_MIN = [5, 10]

# --- Шаг 3: Основной цикл и сбор данных ---

# Главный словарь для хранения всех результатов
all_isochrones_data = {}


checkpoint_counter = 0
CHECKPOINT_INTERVAL = 10

# Внешний цикл по точкам
for index, point in points_df.iterrows():
    lat = point['lat']
    lon = point['lon']

    # Создаем ключ для точки
    point_key = f"{lat},{lon}"
    # --- ИЗМЕНЕНИЕ: Инициализируем пустой словарь ---
    all_isochrones_data[point_key] = {}

    #print(f"\nОбработка точки: {point_key}")

    # Средний цикл по скоростям
    for speed in SPEEDS_KMH:
        speed_key = str(speed)
        all_isochrones_data[point_key][speed_key] = {}

        # Внутренний цикл по времени
        for time_value in ISO_TIMES_MIN:
            time_key = str(time_value)

            #print(f"  -> Расчет для скорости {speed} км/ч, время {time_value} мин...")

            #№start_time = timer.time()
            iso_gdf = create_isochrone(lat, lon, speed, time_value)
            #end_time = timer.time()

            #print(f"     ... готово за {end_time - start_time:.2f} сек.")

            if iso_gdf is not None and not iso_gdf.empty:
                polygon_geometry = iso_gdf.geometry.iloc[0]
                polygon_geojson_dict = polygon_geometry.__geo_interface__
                all_isochrones_data[point_key][speed_key][time_key] = polygon_geojson_dict
            else:
                all_isochrones_data[point_key][speed_key][time_key] = None

    if (index + 1) % CHECKPOINT_INTERVAL == 0 and index > 0:
      checkpoint_counter += 1
      checkpoint_filename = f"isochrone_results_checkpoint_{checkpoint_counter}.json"

      #print(f"\n--- СОЗДАНИЕ ЧЕКПОИНТА #{checkpoint_counter} ---")
      #print(f"Обработано {index + 1} точек. Сохранение промежуточных результатов в {checkpoint_filename}...")

      try:
          with open(checkpoint_filename, 'w', encoding='utf-8') as f:
              json.dump(all_isochrones_data, f, ensure_ascii=False, indent=4)
          #print(f"--- ЧЕКПОИНТ УСПЕШНО СОХРАНЕН ---")
      except Exception as e:
          print(f"--- ОШИБКА СОХРАНЕНИЯ ЧЕКПОИНТА: {e} ---")

In [ ]:
import json

In [ ]:
# --- Шаг 4: Сохранение итогового результата в JSON ---

output_filename = "isochrone_results.json"
print(f"\nСохранение всех результатов в файл: {output_filename}")

with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(all_isochrones_data, f, ensure_ascii=False, indent=4)

print("Готово!")


Сохранение всех результатов в файл: isochrone_results.json
Готово!


In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
# DO NOT TOUCH DO NOT TOUCH DO NOT TOUCH

In [ ]:
pip install --upgrade osmnx

In [ ]:
import osmnx as ox
print(osmnx.__version__)

2.0.5


In [ ]:
def create_isochrone_osmnx(latitude, longitude, walking_speed_kmh, iso_time_min):
    """
    Создает изохрону с помощью встроенной функции OSMnx.

    :param graph: Граф дорожной сети от OSMnx.
    :param latitude: Широта начальной точки.
    :param longitude: Долгота начальной точки.
    :param walking_speed_kmh: Скорость пешехода в км/ч.
    :param iso_time_min: Время доступности в минутах.
    :return: GeoDataFrame с полигоном изохроны.
    """
    print(f"Начальная точка: {latitude}, {longitude}")
    start_point = (latitude, longitude)

    # ВАЖНО: OSMnx сам делает все расчеты, включая построение полигона.
    # travel_times - это список времен в минутах.
    # speed - скорость в км/ч.
    isochrone_gdf = ox.isochrones_from_point(
        G,
        start_point,
        travel_times=[iso_time_min],
        speed=walking_speed_kmh
    )

    # Сохраняем изохрону в GeoJSON
    filepath = f"moscow_{iso_time_min}min_isochrone_{latitude}_{longitude}_osmnx.geojson"
    isochrone_gdf.to_file(filepath, driver="GeoJSON")
    print(f"Изохрона сохранена в: {filepath}")

    return isochrone_gdf


In [ ]:
create_isochrone_osmnx(55.747058, 37.641538, 4.5, 10)

Начальная точка: 55.747058, 37.641538


AttributeError: module 'osmnx' has no attribute 'isochrones_from_point'

In [ ]:
def create_isochrone_2(latitude, longitude, travel_time, speed, network_type='walk'):
    """
    Создает изохрону для заданной точки

    Параметры:
    G - граф дорожной сети
    center_point - кортеж (широта, долгота) стартовой точки
    travel_time - время изохроны в минутах
    speed - скорость движения в км/ч
    network_type - тип сети ('walk', 'bike', 'drive')

    Возвращает:
    GeoDataFrame с полигоном изохроны
    """
    # Находим ближайший узел
    center_node = ox.distance.nearest_nodes(G, latitude, longitude)

    # Рассчитываем время движения для каждого ребра
    for u, v, k, data in G.edges(keys=True, data=True):
        length = data.get('length', 0)  # метры
        # Время в минутах: расстояние / (скорость * 1000/60)
        data['travel_time'] = length / (speed * 1000 / 60)

    # Рассчитываем время пути до всех узлов
    travel_times = nx.single_source_dijkstra_path_length(
        G,
        center_node,
        weight='travel_time'
    )

    # Фильтруем достижимые узлы
    reached_nodes = [
        node for node, time in travel_times.items()
        if time <= travel_time
    ]

    # Создаем точки из достижимых узлов
    points = [
        Point(G.nodes[node]['x'], G.nodes[node]['y'])
        for node in reached_nodes
    ]

    # Создаем полигон (выпуклая оболочка)
    polygon = MultiPoint(points).convex_hull

    # Создаем GeoDataFrame
    gdf = gpd.GeoDataFrame(
        geometry=[polygon],
        crs="EPSG:4326",
        data={'time_min': [travel_time], 'speed_kmh': [speed]}
    )

    return gdf